<a href="https://colab.research.google.com/github/is-leeroy-jenkins/Boo/blob/main/ipynb/GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Dependencies

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from Static import GptRequests, GptRoles, GptLanguages
from Booger import ErrorDialog, Error, FileDialog, SaveFileDialog, FolderDialog
import FileSys as fs
import Minion as mn
import Boo as ai
from openai import OpenAI
from pathlib import Path
import warnings

#### Create chat completion

In [ ]:
# Create Client
client = OpenAI( )
client.api_key = os.getenv( 'OPENAI_API_KEY' )

# Create Completion
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    { "role": "developer", "content": "You are a helpful assistant." },
    { "role": "user", "content": "What is an appropriation?" }
  ]
)

print( completion.choices[ 0 ].message )

#### Get chat completion

In [ ]:
# Create Client
client = OpenAI( )
client.api_key = os.getenv( 'OPENAI_API_KEY' )

# Create Completion
completions = client.chat.completions.list( )
first_id = completions[ 0 ].id
first_completion = client.chat.completions.retrieve( completion_id=first_id )
print( first_completion )

#### Get chat messages

In [ ]:
# Create Client
client = OpenAI( )
client.api_key = os.getenv( 'OPENAI_API_KEY' )

# Create Completion
completions = client.chat.completions.list( )
first_id = completions[ 0 ].id
first_completion = client.chat.completions.retrieve( completion_id=first_id )
messages = client.chat.completions.messages.list( completion_id=first_id )
print( messages )

#### List Chat Completions

In [ ]:
# Create Client
client = OpenAI( )
client.api_key = os.getenv( 'OPENAI_API_KEY' )

# Create Compmletion
completions = client.chat.completions.list()
print(completions)


#### Update chat completion

In [ ]:
# Create Client
client = OpenAI()
client.api_key = os.getenv( 'OPENAI_API_KEY' )

# Create Completion
completions = client.chat.completions.list( )
first_id = completions[0].id
updated_completion = client.chat.completions.update( completion_id=first_id,
	request_body={"metadata": {"foo": "bar"}})
print(updated_completion)

#### Delete chat completion

In [ ]:
# Create Client
client = OpenAI( )
client.api_key = os.getenv( 'OPENAI_API_KEY' )

# Build Completion
completions = client.chat.completions.list( )
first_id = completions[ 0 ].id
delete_response = client.chat.completions.delete( completion_id=first_id )
print( delete_response )

#### Create speech

In [ ]:
speech_file_path = Path(__file__).parent # "speech.mp3"
response = openai.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input="The quick brown fox jumped over the lazy dog."
)
response.stream_to_file(speech_file_path)


#### Create transcription

In [ ]:
# Create Client
client = OpenAI( )
client.api_key = os.getenv( 'OPENAI_API_KEY' )

# Open Audio File 'speech.mp3'
audio_file = open( "speech.mp3", "rb" )
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)


#### Create translation

In [ ]:
client = OpenAI( )
client.api_key = os.getenv( 'OPENAI_API_KEY' )

audio_file = open("speech.mp3", "rb")
transcript = client.audio.translations.create(
  model="whisper-1",
  file=audio_file
)


#### Create Embedding


In [ ]:
# Create Client
client = OpenAI()
client.api_key = os.getenv( 'OPENAI_API_KEY' )

client.embeddings.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter...",
  encoding_format="float"
)
